:#Food Vision 101 - Classification Model of Food data with 101 classes

##1. Feature Extraction Base Model

We will first try to create a model for 10% of the Data then we will scale it up to all data


###Import Libraries and my own helper functions


In [ ]:
!pip install sbhelp==0.0.3
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Sat Nov  5 11:15:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-------------

In [ ]:
from sbhelp.sbhelp import plotAccuracy, plotAccuracyBA, plotAccuracyBA, plotLoss

###Download Data and extract

In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
zipRef = zipfile.ZipFile('101_food_classes_10_percent.zip')
zipRef.extractall()
zipRef.close()

--2022-11-05 11:15:06--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.130.128, 142.250.4.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.130.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  23.1MB/s    in 73s     

2022-11-05 11:16:19 (21.3 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



###Preprocessing and Data Augmentation

In [ ]:
trainDir = "101_food_classes_10_percent/train"
testDir = "101_food_classes_10_percent/test"

IMGSize = (224, 224)
InputShape = (224, 224, 3)
batchSize = 32

trainData10 = tf.keras.preprocessing.image_dataset_from_directory(directory = trainDir,
                                                                  image_size = IMGSize,
                                                                  label_mode = 'categorical',
                                                                  batch_size = batchSize)

testData = tf.keras.preprocessing.image_dataset_from_directory(directory = testDir,
                                                               image_size = IMGSize,
                                                               label_mode = 'categorical',
                                                               batch_size = batchSize,
                                                               shuffle = False)

from tensorflow.keras.layers import experimental

dataAugmentation = tf.keras.Sequential([
    experimental.preprocessing.RandomFlip('horizontal'),
    experimental.preprocessing.RandomRotation(0.2),
    experimental.preprocessing.RandomZoom(0.2),
    experimental.preprocessing.RandomWidth(0.2),
    experimental.preprocessing.RandomHeight(0.2),
],  name = 'dataAugmentation')

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


###Setting Up the base model and freeze its layers

In [ ]:
baseModel = tf.keras.applications.EfficientNetB0(include_top=False)
baseModel.trainable = False #Freeze the layers

from tensorflow.keras import layers
inputs = layers.Input(shape = InputShape, name = 'inputLayer')

x = dataAugmentation(inputs)
x = baseModel(x, training=False) #put the baseModel in inference mode so weights which need to stay frozen, stay frozen
x = layers.GlobalAveragePooling2D(name="GlobalAvePool")(x)
output = layers.Dense(len(trainData10.class_names), activation='softmax', name='outputlayer')(x)

modelv1 = tf.keras.Model(inputs, output)

In [ ]:
modelv1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputLayer (InputLayer)     [(None, 224, 224, 3)]     0         
                                                                 
 dataAugmentation (Sequentia  (None, 224, 224, 3)      0         
 l)                                                              
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 GlobalAvePool (GlobalAverag  (None, 1280)             0         
 ePooling2D)                                                     
                                                                 
 outputlayer (Dense)         (None, 101)               129381    
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-train

###Compile and fit the Model v1 and validate

In [ ]:
modelv1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

historyv1 = modelv1.fit(trainData10, epochs = 5, validation_data=testData, validation_steps=int(0.125*len(testData)))

Epoch 1/5
237/237 [==============================] - 56s 186ms/step - loss: 0.0867 - accuracy: 0.1446 - val_loss: 0.0509 - val_accuracy: 0.3673
Epoch 2/5
237/237 [==============================] - 41s 171ms/step - loss: 0.0459 - accuracy: 0.4007 - val_loss: 0.0419 - val_accuracy: 0.4745
Epoch 3/5
237/237 [==============================] - 43s 177ms/step - loss: 0.0388 - accuracy: 0.5020 - val_loss: 0.0377 - val_accuracy: 0.5226
Epoch 4/5
237/237 [==============================] - 41s 171ms/step - loss: 0.0345 - accuracy: 0.5490 - val_loss: 0.0351 - val_accuracy: 0.5469
Epoch 5/5
237/237 [==============================] - 39s 163ms/step - loss: 0.0316 - accuracy: 0.5823 - val_loss: 0.0335 - val_accuracy: 0.5603


###Evaluate on whole test dataset

In [ ]:
modelv1.evaluate(testData)

790/790 [==============================] - 59s 75ms/step - loss: 0.0335 - accuracy: 0.5720


[0.033459391444921494, 0.5719603896141052]

###Visualizing the training curves

In [ ]:
plotLoss(historyv1)

In [ ]:
plotAccuracy(historyv1)

###Summary

1.   Accuracy 58.63% on Training and 57.33% on Test Data
2.   Training Accuracy curve starts to seperate from Validation Accuary curve after 4 epoch
3.   Training Loss curve starts to seperate from Validation Loss curve after 4 epoch


The plotted curves seems to suggest that after 4 epochs the training and validation curves starts to seperate a bit. Suggesting if we train for more epochs the model will overfit. The turnaround for this is to fine tune the model by unfreezing some layers of the efficient net.


##Fine Tuning the model

###Unfreezing some layers 

Note - Since the feature extraction model performed quite well we will just Unfreeze 5 layers and decrease the learning rate by 10x

In [ ]:
baseModel.trainable = True #Unfreeze all layers

for layer in baseModel.layers[: -5]:
  layer.trainable = False #Refreeze all layers except the last 5 layers

###Re-Compile and fit the baseModel with the revised learning rate and trainable layers

In [ ]:
modelv1.compile(loss =tf.keras.losses.BinaryCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics = ['accuracy'])

fineTuneEpoch = 10

historyFineTune = modelv1.fit(trainData10,
                              epochs = fineTuneEpoch,
                              validation_data = testData,
                              validation_steps = int(0.15*len(testData)),
                              initial_epoch = historyv1.epoch[-1])

Epoch 5/10
237/237 [==============================] - 50s 184ms/step - loss: 0.0261 - accuracy: 0.6242 - val_loss: 0.0267 - val_accuracy: 0.5736
Epoch 6/10
237/237 [==============================] - 39s 162ms/step - loss: 0.0233 - accuracy: 0.6405 - val_loss: 0.0257 - val_accuracy: 0.5776
Epoch 7/10
237/237 [==============================] - 40s 165ms/step - loss: 0.0217 - accuracy: 0.6635 - val_loss: 0.0252 - val_accuracy: 0.5824
Epoch 8/10
237/237 [==============================] - 39s 161ms/step - loss: 0.0205 - accuracy: 0.6849 - val_loss: 0.0247 - val_accuracy: 0.5861
Epoch 9/10
237/237 [==============================] - 39s 165ms/step - loss: 0.0197 - accuracy: 0.6972 - val_loss: 0.0246 - val_accuracy: 0.5895
Epoch 10/10
237/237 [==============================] - 38s 159ms/step - loss: 0.0189 - accuracy: 0.7114 - val_loss: 0.0242 - val_accuracy: 0.5922


###Evaluate on whole test data

In [ ]:
modelv1.evaluate(testData)

790/790 [==============================] - 58s 73ms/step - loss: 0.0238 - accuracy: 0.6060


[0.023773975670337677, 0.6059802174568176]

###Visualizing the training curves

In [ ]:
plotAccuracyBA(historyv1, historyFineTune)

###Summary

1.   The after fine tuning curves are elevated above 4%
2.   Training and validation curves seperate drastically

As our fine tuning curves are elevated, it means that the fine tuning will perform better than the base model in full dataset. Also it appears that the model is overfitting.



###Saving the model

In [ ]:
modelv1.save('drive/MyDrive/MyModels/101FoodVision')

In [ ]:
loadedModel = tf.keras.models.load_model('drive/MyDrive/MyModels/101FoodVision')

In [ ]:
loadedModel.evaluate(testData)

790/790 [==============================] - 71s 76ms/step - loss: 0.0238 - accuracy: 0.6060


[0.023773998022079468, 0.6059802174568176]

In [ ]:
predProbs = loadedModel.predict(testData, verbose=1)

790/790 [==============================] - 65s 82ms/step


In [ ]:
predClasses = predProbs.argmax(axis=1)

In [ ]:
yLabels = []

for images, labels in testData.unbatch():
  yLabels.append(labels.numpy().argmax())

yLabels[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
predClasses[:10]

array([ 2,  0,  0,  8,  8, 78, 29,  0,  2,  0])

In [ ]:
from sklearn.metrics import accuracy_score

sklearnAcc = accuracy_score(yLabels, predClasses)

In [ ]:
sklearnAcc

0.605980198019802